# PomoDock Prototyping Notebook

## Weather Module

In [3]:
import requests
import json

# Get weather data for current time and for a 5 hour window.
# Current data: Air temperature at 2 meters above ground, Apparent Weather or "feels like", weather code (Weather condition as a numeric code. i.e. 0 = Clear Sky, 51 = light drizzle , 95 = Thunderstorm, etc.)
# 5 hour window data: Air temperature at 2 meters above ground, weather code. These two make most sense for upcoming hours.
# ^ likely will increase to 6 and shift off 1st element/"current" hour. This should be dynamic in the feature for users to configure this.

# TODO:
# - refactor this line to include enums/constant variables
# - Prettify response
# - Prototype forecast output

# Build param payload
param_payload = {'temperature_unit': 'fahrenheit',
                 # Busan Coordinates
                 'latitude': '35.166668',
                 'longitude': '129.066666',
                 'timezone': 'CST',
                 'current':['temperature_2m','apparent_temperature','weather_code'],
                 'hourly': ['temperature_2m','weather_code','is_day'],
                 'forecast_hours': '5'}

# GET request, stuffs data into response
response = requests.get("https://api.open-meteo.com/v1/forecast", params=param_payload)

In [4]:
# Pretty print JSON object
print(json.dumps(response.json(), indent=4))

{
    "latitude": 35.15,
    "longitude": 129.0625,
    "generationtime_ms": 0.09918212890625,
    "utc_offset_seconds": -21600,
    "timezone": "CST",
    "timezone_abbreviation": "GMT-6",
    "elevation": 32.0,
    "current_units": {
        "time": "iso8601",
        "interval": "seconds",
        "temperature_2m": "\u00b0F",
        "apparent_temperature": "\u00b0F",
        "weather_code": "wmo code"
    },
    "current": {
        "time": "2025-11-11T22:45",
        "interval": 900,
        "temperature_2m": 62.8,
        "apparent_temperature": 60.2,
        "weather_code": 1
    },
    "hourly_units": {
        "time": "iso8601",
        "temperature_2m": "\u00b0F",
        "weather_code": "wmo code",
        "is_day": ""
    },
    "hourly": {
        "time": [
            "2025-11-11T22:00",
            "2025-11-11T23:00",
            "2025-11-12T00:00",
            "2025-11-12T01:00",
            "2025-11-12T02:00"
        ],
        "temperature_2m": [
            62.7,
   

### WMO Weather Interpretation Codes

```
Code	    Description

0	    Clear sky
1, 2, 3	    Mainly clear, partly cloudy, and overcast
45, 48	    Fog and depositing rime fog
51, 53, 55  Drizzle: Light, moderate, and dense intensity
56, 57	    Freezing Drizzle: Light and dense intensity
61, 63, 65  Rain: Slight, moderate and heavy intensity
66, 67      Freezing Rain: Light and heavy intensity
71, 73, 75  Snow fall: Slight, moderate, and heavy intensity
77          Snow grains
80, 81, 82  Rain showers: Slight, moderate, and violent
85, 86      Snow showers slight and heavy
95 *        Thunderstorm: Slight or moderate
96, 99 *    Thunderstorm with slight and heavy hail

(*) Thunderstorm forecast with hail is only available in Central Europe
```

In [5]:
# Weather codes in dict format
weather_codes = {
    0: "Clear sky",
    1: "Mostly clear",
    2: "Partly cloudy",
    3: "Overcast",
    45: "Fog",
    48: "Rime fog (icy fog)",
    51: "Light drizzle",
    53: "Moderate drizzle",
    55: "Heavy drizzle",
    56: "Light freezing drizzle",
    57: "Heavy freezing drizzle",
    61: "Light rain",
    63: "Moderate rain",
    65: "Heavy rain",
    66: "Light freezing rain",
    67: "Heavy freezing rain",
    71: "Light snowfall",
    73: "Moderate snowfall",
    75: "Heavy snowfall",
    77: "Snow grains",
    80: "Light rain showers",
    81: "Moderate rain showers",
    82: "Heavy rain showers",
    85: "Light snow showers",
    86: "Heavy snow showers",
    95: "Thunderstorm",
    96: "Thunderstorm with light hail",
    99: "Thunderstorm with heavy hail"
}

In [6]:
response_json = response.json()
# Show current conditions snapshot
print('Busan')
print(f"Current Temperature: {response_json.get("current").get('temperature_2m')}")
print(f"Feels Like: {response_json.get("current").get('apparent_temperature')}")

Busan
Current Temperature: 62.8
Feels Like: 60.2


In [26]:
from datetime import datetime

def convertIsoTime(isoString):
    date_time = datetime.fromisoformat(isoString)
    readable_time = date_time.strftime("%-I:%M %p")
    return readable_time

def formatTemp(temp, unit):
    return f"{temp} {unit}"

def getDayNight(isDay):
    return "Daytime ☀️" if isDay else "Nighttime 🌙"

def showHourlyForecast(hourly_data, hourly_units):
    hourly_time = hourly_data.get("time")
    hourly_temps = hourly_data.get("temperature_2m")
    hourly_weather_code = hourly_data.get("weather_code")
    hourly_is_day = hourly_data.get("is_day")

    number_of_hours = len(hourly_time)
    for hour in range(number_of_hours):
        print(convertIsoTime(hourly_time[hour]))
        print(formatTemp(hourly_temps[hour],hourly_units.get("temperature_2m")))
        print(weather_codes.get(hourly_weather_code[hour]))
        print(getDayNight(hourly_is_day[hour]))
        if hour != number_of_hours - 1:
            print()
        
hourly_data = response_json.get("hourly")
hourly_units = response_json.get("hourly_units")
showHourlyForecast(hourly_data, hourly_units)

10:00 PM
62.7 °F
Partly cloudy
Daytime ☀️

11:00 PM
62.8 °F
Mostly clear
Daytime ☀️

12:00 AM
62.3 °F
Partly cloudy
Daytime ☀️

1:00 AM
61.6 °F
Overcast
Daytime ☀️

2:00 AM
60.3 °F
Partly cloudy
Daytime ☀️


## Pomodoro Timer Module
The Pomodoro Technique is a time management method that helps improve focus and productivity by breaking work into short, timed intervals called Pomodoros, separated by brief breaks. It’s designed to keep your mind fresh, prevent burnout, and make progress feel tangible.

In [25]:
import time

SECONDS_IN_A_MINUTE = 60

def start_timer(duration_minutes):
    duration_seconds = duration_minutes * SECONDS_IN_A_MINUTE
    # Capture the timestamp the timer was started 
    start_time = time.time()
    displayed_minutes_left = duration_minutes
    # While the current time - start_time (AKA the time elapsed) is less than the duration,
    # print the time remaining in minutes
    while (time.time() - start_time) < duration_seconds:
        whole_minutes_left = int(duration_minutes - ((time.time() - start_time) // 60))
        
        if whole_minutes_left < displayed_minutes_left:
            displayed_minutes_left = whole_minutes_left
            print(f"{displayed_minutes_left} minutes left")
        time.sleep(1)
    print("time is up!")

# Start timer to track 3 minutes for dev
start_timer(3)

2 minutes left
1 minutes left
time is up!


## Display Output Generation

The vision is to have an eink display as the main output for this device. Since eink displays are not the type to have dynamic or reactive content displayed on them, it makes most sense that content is generated in forms that are the most compatible for them: Images.

I'm most comfortbale with technologies like HTML/CSS, so it sounds like a good option to construct the UI elements with HTML and style them via CSS as usual. Then, we can go ahead and use something like Pyppeteer to open the html file in a headless browser and take a screenshot of the page. This will effectively create an image that can be easily displayed on an eink display.

This is what ties everything together for the most part. I am thinking that this methodology can be applied to update the weather data periodically, and also be what's used to update the display to portray how many minutes are left on the pomodoro timer aspect of the device.

In [4]:
# Read in HTML template
with open("./templates/WeatherDashboard.html", "r") as f:
    html = f.read()

# Fill in template placeholders
html_filled = (
    html.replace("{{currentTemp}}", str(43))
        .replace("{{currentCondition}}", "Cloudy")
        .replace("{{currentFeelsLike}}", str(41))
)

# Write out filled HTML
with open("./templateRenders/rendered.html", "w") as f:
    f.write(html_filled)

In [2]:
import asyncio
from pyppeteer import launch

async def html_to_png(html_file, output_file):
    browser = await launch(headless=True, args=[
        "--no-sandbox",  # optional depending on Pi
        "--disable-dev-shm-usage"
    ])

    page = await browser.newPage()
    await page.setViewport({"width": 600, "height": 448})  # your e-ink size

    await page.goto(f"file://{html_file}")
    await page.screenshot(path=output_file, fullPage=True)

    await browser.close()

# To run it:
# asyncio.get_event_loop().run_until_complete(
    html_to_png("./templateRenders/rendered.html", "./display/output.png")
# )